# Logistic Regression Visualization with Manim

This notebook demonstrates gradient descent optimization for logistic regression.

In [ ]:
from manim import BOLD
from mymanim import *
import numpy as np
config.frame_width = 25
config.frame_height = 10

## Logistic Regression Animation Class

In [ ]:
class LogisticRegressionDemo(Scene):
    def construct(self):
        # 1. Generate two classes of 2D points (linearly separable)
        np.random.seed(0)
        n_points = 20
        X1 = np.random.randn(n_points, 2) * 1.0 + np.array([2, 2])  # Class 1 points
        X2 = np.random.randn(n_points, 2) * 1.0 + np.array([6, 6])  # Class 2 points
        X = np.vstack([X1, X2])
        y = np.array([0]*n_points + [1]*n_points)

        # 2. Create axes
        axes = Axes(
            x_range=[-2, 10, 1],
            y_range=[-2, 10, 1],
            axis_config={"color": WHITE},
            x_length=10,
            y_length=10,
        ).move_to(LEFT*4)  # Move axes to the left
        self.play(Create(axes))

        # 3. Plot data points
        dots1 = VGroup(*[Dot(axes.c2p(*pt), color=BLUE) for pt in X1])  # Class 0 points
        dots2 = VGroup(*[Dot(axes.c2p(*pt), color=RED) for pt in X2])   # Class 1 points
        self.play(FadeIn(dots1, lag_ratio=0.1), FadeIn(dots2, lag_ratio=0.1))
        self.wait(0.5)

        # 4. Initialize decision boundary parameters
        w_tracker = ValueTracker(1)    # w1 coefficient
        v_tracker = ValueTracker(-1)   # w2 coefficient
        b_tracker = ValueTracker(0)    # bias term

        # 5. Create decision boundary line
        def get_line():
            w1 = w_tracker.get_value()
            w2 = v_tracker.get_value()
            b0 = b_tracker.get_value()
            if abs(w2) < 1e-3:
                w2 = 1e-3
            if abs(w1) < 1e-3:
                w1 = 1e-3
            def y_func(x): return -(w1/w2)*x - b0/w2
            x_vals = []
            for x in np.linspace(0, 8, 200):
                y = y_func(x)
                if 0 < y < 8:
                    x_vals.append(x)
            # Calculate valid range
            if len(x_vals) < 2:
                return VGroup()
            length = x_vals[-1] - x_vals[0]
            # Fade line when too short
            if length < 0.5:
                alpha = max(0.1, length / 0.5)
            else:
                alpha = 1.0
            return axes.plot(y_func, color=YELLOW, x_range=[x_vals[0], x_vals[-1]], stroke_opacity=alpha)
        
        sep_line = always_redraw(get_line)
        self.add(sep_line)

        # Helper functions for logistic regression
        def sigmoid(z):
            return 1/(1+np.exp(-z))
            
        def get_loss():
            w1 = w_tracker.get_value()
            w2 = v_tracker.get_value()
            b0 = b_tracker.get_value()
            z = w1*X[:,0] + w2*X[:,1] + b0
            y_pred = sigmoid(z)
            loss = -np.mean(y*np.log(y_pred+1e-8)+(1-y)*np.log(1-y_pred+1e-8))
            return loss

        # Track loss history
        loss_history = ValueTracker(get_loss())

        # Counter for unchanged loss
        unchanged_count = [0]
        last_loss = [get_loss()]

        def get_loss_color():
            curr_loss = get_loss()
            if abs(curr_loss - last_loss[0]) < 1e-8:
                unchanged_count[0] += 1
            else:
                unchanged_count[0] = 0
            last_loss[0] = curr_loss
            # Turn yellow if loss hasn't changed for 10 steps
            if unchanged_count[0] >= 10:
                return YELLOW
            else:
                return WHITE

        # Dynamic loss display below axes
        loss_text = always_redraw(lambda: Text(
            f"Current Loss: {get_loss():.3f}", font_size=44, color=get_loss_color(), weight=BOLD
        ).next_to(axes, DOWN, buff=0.5))
        self.add(loss_text)

        # Learning rate display
        lr = 0.1
        lr_text = always_redraw(lambda: Text(
            f"Learning Rate: {lr}", font_size=44, color=WHITE, weight=BOLD
        ).next_to(axes, UP, buff=0.5))
        self.add(lr_text)

        # Loss curve visualization
        steps = 200
        loss_history_list = []
        max_loss = get_loss()
        loss_axes = Axes(
            x_range=[0, steps, 10],
            y_range=[0, max_loss*1.5 if max_loss > 0 else 1.5, max_loss/5 if max_loss > 0 else 0.3],
            x_length=8,
            y_length=4,
        )
        loss_axes.next_to(axes, RIGHT, buff=1.2)
        loss_axes.align_to(axes, DOWN)
        loss_axes.shift(UP*1.3)
        
        # Axis labels
        x_label = Text("Training Steps", font_size=28)
        x_label.next_to(loss_axes.x_axis, DOWN, buff=0.3)
        y_label = Text("Loss", font_size=28)
        y_label.next_to(loss_axes.y_axis, LEFT, buff=0.3)
        y_label.align_to(loss_axes.y_axis, UP)
        y_label.shift(UP*0.5)
        self.add(loss_axes, x_label, y_label)
        
        def get_loss_curve():
            if len(loss_history_list) < 2:
                return VGroup()
            x_vals = np.arange(len(loss_history_list))
            y_vals = np.array(loss_history_list)
            return loss_axes.plot_line_graph(
                x_values=x_vals,
                y_values=y_vals,
                add_vertex_dots=False,
                line_color=YELLOW
            )
        
        loss_curve = always_redraw(get_loss_curve)
        self.add(loss_curve)

        # 8. Gradient descent animation
        for i in range(steps):
            w1 = w_tracker.get_value()
            w2 = v_tracker.get_value()
            b0 = b_tracker.get_value()
            
            # Compute gradients
            z = w1*X[:,0] + w2*X[:,1] + b0
            y_pred = sigmoid(z)
            grad_w1 = np.mean((y_pred - y) * X[:,0])
            grad_w2 = np.mean((y_pred - y) * X[:,1])
            grad_b = np.mean(y_pred - y)
            
            # Update parameters
            new_w1 = w1 - lr * grad_w1
            new_w2 = w2 - lr * grad_w2
            new_b = b0 - lr * grad_b
            
            self.play(
                w_tracker.animate.set_value(new_w1),
                v_tracker.animate.set_value(new_w2),
                b_tracker.animate.set_value(new_b),
                run_time=0.035
            )
            
            if i == 0:
                self.wait(0.2)
                
            # Record loss history
            loss_history_list.append(get_loss())
        
        self.wait(0.5)

        # 9. Show final loss in yellow
        self.remove(loss_text)
        final_loss_text = Text(
            f"Final Loss: {get_loss():.3f}", font_size=44, color=YELLOW, weight=BOLD
        ).next_to(axes, DOWN, buff=0.5)
        self.add(final_loss_text)
        self.wait(5)

## Running the Animation

In [ ]:
# To render the animation in Jupyter, you would typically use:
# %manim -v WARNING -qh LogisticRegressionDemo

# Note: Requires Manim and Jupyter integration to be properly set up